In [27]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.callbacks import Callback

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
data_makanan = pd.read_csv('/content/drive/Shared drives/Dataset/Data Makanan Indonesia.csv')
data_pasien_stroke = pd.read_csv('/content/drive/Shared drives/Dataset/Data Training.csv')
print(data_pasien_stroke)

        id  gender age  hypertension  heart_disease  avg_glucose_level   bmi  \
0    41182  Female  35             1              0              94.20  34.4   
1    71424  Female  75             1              0              55.96  34.8   
2      259    Male  79             0              0             198.79  24.9   
3    69112    Male  68             1              1             271.74  31.1   
4    11539  Female  24             1              0             107.22  35.3   
..     ...     ...  ..           ...            ...                ...   ...   
178  10538    Male  75             1              1             195.03  28.7   
179  38575    Male  58             1              0             209.15  52.9   
180  17771  Female  64             1              0             211.12  22.0   
181  37253  Female  70             1              0             147.12  22.3   
182  66315  Female  57             0              0              68.02  37.5   

    food_code  
0       CP015  
1      

PREPROSESSING

In [4]:
data_pasien_stroke_baru = data_pasien_stroke.dropna()

data_pasien_stroke_baru['avg_glucose_level'] = data_pasien_stroke_baru['avg_glucose_level'].astype(int)
for index, row in data_pasien_stroke_baru.iterrows():
    if row['avg_glucose_level'] > 100:
        data_pasien_stroke_baru.at[index, 'avg_glucose_level'] = 1
    else:
        data_pasien_stroke_baru.at[index, 'avg_glucose_level'] = 0
data_pasien_stroke_baru.dtypes


id                     int64
gender                object
age                   object
hypertension           int64
heart_disease          int64
avg_glucose_level      int64
bmi                  float64
food_code             object
dtype: object

Memisahkan Fitur

In [5]:
feature = data_pasien_stroke_baru[['hypertension', 'heart_disease','avg_glucose_level','bmi']].values
target = data_pasien_stroke_baru['food_code'].values

Normalisasi Data

In [6]:
# Normalisasi fitur menggunakan StandardScaler
scaler = StandardScaler()
feature = scaler.fit_transform(feature)

Splitting Data Train dan Validation

In [9]:
X_train=feature
Y_train=target
label_encoder = LabelEncoder()
Y_train_encoded = label_encoder.fit_transform(Y_train)

In [29]:
class SaveBestAccuracyModel(Callback):
    def __init__(self, filepath, monitor='val_acc', mode='max', save_best_only=True):
        super(SaveBestAccuracyModel, self).__init__()
        self.filepath = filepath
        self.monitor = monitor
        self.mode = mode
        self.save_best_only = save_best_only
        self.best_accuracy = -float('inf') if mode == 'max' else float('inf')

    def on_epoch_end(self, epoch, logs=None):
        current_accuracy = logs.get(self.monitor)
        if current_accuracy is None:
            print("SaveBestAccuracyModel warning: Accuracy metric '{}' not found. Model won't be saved.".format(self.monitor))
            return

        if (self.mode == 'max' and current_accuracy > self.best_accuracy) or \
           (self.mode == 'min' and current_accuracy < self.best_accuracy):

            print("\nSaveBestAccuracyModel: Accuracy improved from {:.4f} to {:.4f}. Saving model to {}"
                  .format(self.best_accuracy, current_accuracy, self.filepath))
            self.best_accuracy = current_accuracy
            self.model.save(self.filepath, overwrite=True)
        elif self.save_best_only and epoch > 0:
            print("\nSaveBestAccuracyModel: Accuracy did not improve from {:.4f}. Model not saved.".format(self.best_accuracy))

Model Machine Learning Artificial Neural Network (Content Based Filtering)

In [30]:
# Membangun model Neural Network

model = tf.keras.Sequential([
    tf.keras.layers.Dense(8, activation='relu', input_shape=X_train.shape[1:]),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(len(np.unique(Y_train_encoded)), activation='softmax')
])
# Mengompilasi model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
#Membuat model dengan hasil terbaik
callbacks = [SaveBestAccuracyModel(filepath='Model_Rekomendasi.h5', monitor='accuracy', mode='max', save_best_only=True)]

# Melatih model dengan menggunakan callback
model.fit(X_train, Y_train_encoded, epochs=1000, callbacks=[callbacks])

Epoch 1/1000
1/6 [====>.........................] - ETA: 12s - loss: 4.5709 - accuracy: 0.0000e+00
SaveBestAccuracyModel: Accuracy improved from -inf to 0.0164. Saving model to Model_Rekomendasi.h5
6/6 [==============================] - 2s 13ms/step - loss: 4.5710 - accuracy: 0.0164
Epoch 2/1000
1/6 [====>.........................] - ETA: 0s - loss: 4.5376 - accuracy: 0.0625
SaveBestAccuracyModel: Accuracy improved from 0.0164 to 0.0383. Saving model to Model_Rekomendasi.h5
6/6 [==============================] - 0s 14ms/step - loss: 4.5474 - accuracy: 0.0383
Epoch 3/1000
1/6 [====>.........................] - ETA: 0s - loss: 4.5278 - accuracy: 0.0938
SaveBestAccuracyModel: Accuracy improved from 0.0383 to 0.0546. Saving model to Model_Rekomendasi.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


6/6 [==============================] - 0s 14ms/step - loss: 4.5229 - accuracy: 0.0546
Epoch 4/1000
1/6 [====>.........................] - ETA: 0s - loss: 4.5495 - accuracy: 0.0312
SaveBestAccuracyModel: Accuracy improved from 0.0546 to 0.0710. Saving model to Model_Rekomendasi.h5
6/6 [==============================] - 0s 13ms/step - loss: 4.4928 - accuracy: 0.0710
Epoch 5/1000
1/6 [====>.........................] - ETA: 0s - loss: 4.4789 - accuracy: 0.0938
SaveBestAccuracyModel: Accuracy did not improve from 0.0710. Model not saved.
6/6 [==============================] - 0s 6ms/step - loss: 4.4559 - accuracy: 0.0546
Epoch 6/1000
1/6 [====>.........................] - ETA: 0s - loss: 4.4233 - accuracy: 0.0312
SaveBestAccuracyModel: Accuracy did not improve from 0.0710. Model not saved.
6/6 [==============================] - 0s 6ms/step - loss: 4.4055 - accuracy: 0.0437
Epoch 7/1000
1/6 [====>.........................] - ETA: 0s - loss: 4.3840 - accuracy: 0.0625
SaveBestAccuracyModel: Ac

# New Section